In [1]:
# torch imports
import torch
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
# Local imports
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml defaults_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    hard_pred_threshold: 0.5 

log:
    root: '?'
    log_interval: 10 
    log_image_stats: True 
    log_pixel_stats: False 
    gether_inference_stats: True
    summary_compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: '?' 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: '?'

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: '?' 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
data:
    preload: False 

model:
    _type: "standard"
    pretrained_exp_root : None
    pretrained_select_metric: "val-dice_score"
    checkpoint: '?'
    pred_label: '?' 

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "06_24_24_DRIVE_LossFunc_Analysis"

base_models: 
    - "/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_CE"
    - "/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_Combo"
    - "/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_Dice"
    - "/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_SVLS_CE"
    - "/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_SVLS_Combo"
    - "/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_SVLS_Dice"

data:
    _class: "ese.data.datasets.DRIVE"
    split: ("val", "cal")
    label_threshold: 0.5

<IPython.core.display.Javascript object>

In [6]:
# Get the configs for the different runs.
base_cfg = Config(defaults_cfg).update([calibration_cfg, model_cfg])
# Get the different experiment cfg yamls.
inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

{'name': ['06_24_24_DRIVE_LossFunc_Analysis'], 'base_models': [('/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_CE', '/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_Combo', '/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_Dice', '/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_SVLS_CE', '/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_SVLS_Combo', '/storage/vbutoi/scratch/ESE/training/06_24_24_DRIVE_Large_DRIVE_SVLS_Dice')], 'data.split': ['("val", "cal")'], 'data.label_threshold': [0.5]}


ValueError: 

In [ ]:
len(inf_cfgs)

## Running Jobs

In [ ]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    # run_name=exp_name,
    show_examples=True,
    gpu='1',
) 

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
# )